In [2]:
## import auto reload
%load_ext autoreload
%autoreload 2

from simulation import run_simulation, initialize_simulation


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
num_employees = 100
num_teams = 10
min_team_size = 8
num_friendships = 5
num_spaces = 20
space_types = ["Workstation"] * 10 + ["Meeting Room"] * 5 + ["Break Area"] * 3 + ["Quiet Area"] * 2
space_capacities = [5] * 10 + [10] * 5 + [15] * 3 + [8] * 2

emps, spaces, team_graph, friendship_graph = initialize_simulation(num_employees, num_teams, min_team_size,
                                                                   num_friendships, num_spaces, space_types, space_capacities)

run_simulation(emps, spaces, 10, initial_info_holder=1, team_graph=team_graph, friendship_graph=friendship_graph)


Time step 1
Informed employees: 3/100 (3.00%)
Space 1 (Workstation): [26, 48, 61, 64, 65, 94]
Space 2 (Workstation): [11, 59, 82]
Space 3 (Workstation): [16, 23, 40, 52, 55, 91, 95, 96, 99]
Space 4 (Workstation): [32, 49, 98]
Space 5 (Workstation): [2, 34, 46, 62, 73, 89]
Space 6 (Workstation): [10, 14, 22, 43, 72]
Space 7 (Workstation): [18, 51, 58, 83, 92]
Space 8 (Workstation): [5, 7, 15, 70]
Space 9 (Workstation): [31, 76, 78, 80, 84]
Space 10 (Workstation): [3, 4, 12, 19, 66, 67, 69, 75, 93]
Space 11 (Meeting Room): [63, 87]
Space 12 (Meeting Room): [6, 45]
Space 13 (Meeting Room): [13, 21, 25, 29, 53, 88]
Space 14 (Meeting Room): [37]
Space 15 (Meeting Room): [1, 9, 47, 71]
Space 16 (Break Area): [39]
Space 17 (Break Area): [17, 42, 57, 60, 100]
Space 18 (Break Area): [20, 44, 56, 74, 77, 79, 81, 85]
Space 19 (Quiet Area): [27, 30, 33, 35, 50, 68, 97]
Space 20 (Quiet Area): [8, 24, 28, 36, 38, 41, 54, 86, 90]
Information spread: [1, 9, 71]

Time step 2
Informed employees: 6/100 (

In [14]:
sum(space_capacities)

111

In [9]:
spaces[0].occupants